## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.


План работы: 

1. Загрузка и изучение данных.
2. Предобработка данных. 
3. Исследовательский анализ
4. Вывод.

Описание данных:


   - *children* — количество детей в семье
   - *days_employed* — общий трудовой стаж в днях
   - *dob_years* — возраст клиента в годах
   - *education* — уровень образования клиента
   - *education_id* — идентификатор уровня образования
   - *family_status* — семейное положение
   - *family_status_id* — идентификатор семейного положения
   - *gender* — пол клиента
   - *income_type* — тип занятости
   - *debt* — имел ли задолженность по возврату кредитов
   - *total_income* — ежемесячный доход
   - *purpose* — цель получения кредита

### 1. Откройте файл с данными и изучите общую информацию. 

In [1]:
import pandas as pd
df = pd.read_csv('/datasets/data.csv')
df.info()
df.head(10)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


### Вывод

Каждая строка таблицы содержит информацию о количестве детей, семейном положении, , трудвом стаже в днях, возрасте, уровне образования, семейном положении, гендерной идентификации, зарплате, задолженности и цели кредита.
Названия столбцов удовлетворительны: без пробелов, использована латиница.


###  2. Предобработка данных

### Обработка пропусков

In [2]:
df['education'] = df['education'].str.lower() # приведение строк в столбце к нижнему регистру

df = df[(df['children']>=0) & (df['children']<=10)] # Убраны данные с артефатками в столбце "children" 
df = df[(df['dob_years']> 18) & (df['dob_years']<75) ] # убран возраст, которым не выдают кредиты

df['days_employed'] = df['days_employed'].abs() # Заменены отрицательные значения при помощи модуля
df['years_employed'] = df['days_employed'] / 365 #Добавил столбец с количеством лет стажа

days_employed_avg = df[((df['dob_years']-14)-df['years_employed'])>=0]['days_employed'].median() #медианное знач. выборки дней,
#где стаж реален
years_employed_avg = df[((df['dob_years']-14)-df['years_employed'])>=0]['years_employed'].median() # --||-- выборка лет
total_income_avg = df[((df['dob_years']-14)-df['years_employed'])>=0]['total_income'].median() # --//-- выборка зарплат
df['total_income'] = df['total_income'].fillna(total_income_avg)
df['days_employed'] = df['days_employed'].fillna(days_employed_avg)
df['years_employed'] = df['years_employed'].fillna(years_employed_avg)
df.isna().sum() # Обработаны пропуски


children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
years_employed      0
dtype: int64

### Вывод

Работа с артефактами:
1. Детей не может быть меньше нуля. Также у 76 семей обнаружилось аж 20 детей, при этом количество семей, у которых меньше 20 и более 5 детей, равно 0. Данные удалены, так как процент данных семей менее 1 %.
2. Стаж не может иметь отрицательное количество дней. Применил математичский модуль, чтобы избавиться от минуса.


Пустые значения имелись в двух столбцах: 'days_employed' и 'total_income'. Обнаружена закономерность, что если доход отсутвует, то информации о стаже также нет. Пропуски были заменены на медианное значение.

### Замена типа данных

In [3]:
df['days_employed'] = df['days_employed'].astype('int')

### Вывод

Заменен вещественный тип данных на целочисленный в столбце 'days_employed', так как общий трудовой стаж в днях
нет смысла считать не целыми числами

### Обработка дубликатов

In [4]:
df.duplicated().sum()
df = df.drop_duplicates().reset_index(drop = True)
df.duplicated().sum()

0

### Вывод

Дубликаты могли появиться вследствие сбоя в записи данных. Стоит обратить внимание и разобраться с причинами появления такого «информационного мусора».

### Лемматизация

In [5]:
import pymystem3
from collections import Counter

sentences = df['purpose'].unique() # Найдены неповторяющиеся значения при помощи unique, чтобы не перебирать каждую
#строку отдельно


stemmer = pymystem3.Mystem()
#Создадим цикл, для лемматизации и дальнешего подсчёта каждого элемента
lemmas_list = []
for element in sentences:
    lemma1 = stemmer.lemmatize(element)
    lemmas_list = lemmas_list + lemma1
    
print(Counter(lemmas_list))



Counter({' ': 59, '\n': 38, 'покупка': 10, 'недвижимость': 10, 'автомобиль': 9, 'образование': 9, 'жилье': 7, 'с': 5, 'операция': 4, 'на': 4, 'свой': 4, 'свадьба': 3, 'строительство': 3, 'получение': 3, 'высокий': 3, 'дополнительный': 2, 'для': 2, 'коммерческий': 2, 'жилой': 2, 'заниматься': 2, 'сделка': 2, 'приобретение': 1, 'сыграть': 1, 'проведение': 1, 'семья': 1, 'собственный': 1, 'подержать': 1, 'со': 1, 'подержанный': 1, 'профильный': 1, 'сдача': 1, 'ремонт': 1})


### Вывод

Наибольшее повторение лемы: 'покупка', 'недвижимость', 'автомобиль', 'образование', 'свадьба'. Полученные данные помогут при создании словаря и категоризации данных.

### Категоризация данных

In [6]:

def create_lemmas(sentence):
    """
    Функция возвращает список лемм для предложения sentence
    Предложение будет переведено в нижний регистр
    """
    return stemmer.lemmatize(sentence.lower())

# Словарь ключевых слов и их категорий.

category = {
    'жилье': 'недвижимость',
    'недвижимость': 'недвижимость',
    'автомобиль': 'автомобиль',
    'образование': 'образование',
    'свадьба': 'свадьба'
}
purpose_categ = []
# Идем по списке предложений
for sentence in sentences:
    # и создаем список лемм для каждого из них.
    lemmas = create_lemmas(sentence.lower())
    # Теперь идем по списку лемм и ищем их принадлежность к той или иной категории
    for lemma in lemmas:
        if lemma in category.keys():
            purpose_categ.append(category[lemma])
            #print('Категория для "{}" это "{}"'.format(sentence, category[lemma]))
            # и прекращаем перебирать леммы
            break
    # если цикл не прервался принудительно (через break)
    else:
        # Если нет категории в словаре, то присваивается значение 'прочее'
        purpose_categ.append('прочее')

# Создаётся таблица с категориями, которые соотнесены с лемматизированными причинами
category = pd.DataFrame({'purpose':sentences}) 
category['purpose_cat'] = purpose_categ 
df = df.merge(category, on='purpose')   
df.tail(5)



,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employed,purpose_cat
21225,1,379878,52,среднее,1,женат / замужем,0,F,пенсионер,0,92997.470761,заняться высшим образованием,1040.761664,образование
21226,0,1629,67,высшее,0,женат / замужем,0,F,пенсионер,0,151156.766892,заняться высшим образованием,4.465748,образование
21227,3,3039,26,высшее,0,женат / замужем,0,F,компаньон,0,302642.330826,заняться высшим образованием,8.327818,образование
21228,0,1117,47,среднее,1,гражданский брак,1,F,сотрудник,0,231833.290488,заняться высшим образованием,3.061069,образование
21229,0,184,34,среднее,1,гражданский брак,1,F,сотрудник,0,90529.732781,заняться высшим образованием,0.505890,образование


### Вывод

Категоризация данных проведена по созданному словарю из лемм столбца 'причин', которые наиболее часто встречаются в столбце 'purpose'

### 3. Исследовательский анализ

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [7]:
df_children = df.loc[:,['children', 'debt']] # создание таблицы со столбцами детей и задолженности
df_children['child'] = df_children['children']>0  #создание столбца о наличии детей

df_child1 = df_children.pivot_table(index='children', values='debt',)#проверка зависимости при помощи pivot_table и среднего зн.
df_child2 = df_children.pivot_table(index='child', values='debt',)
print(df_child1, df_child2)

              debt
children          
0         0.075458
1         0.092028
2         0.095145
3         0.082317
4         0.097561
5         0.000000            debt
child          
False  0.075458
True   0.092385


### Вывод

При отсутсвии детей, кредит возвращался на 2% больше, чем когда они имеются

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [8]:

df.pivot_table(index='family_status', values='debt')

,debt
family_status,
Не женат / не замужем,0.097842
в разводе,0.071247
вдовец / вдова,0.065608
гражданский брак,0.093119
женат / замужем,0.075575


### Вывод

У людей, состоящих или состоявших в браке, процент возврата кредита в срок выше, чем у не женатых/ не замужных

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [9]:
df_income = df.loc[:,['total_income', 'debt']]
df_income['low'] = df_income['total_income']<50000  # поделили на низкий, средний и высокий уровень дохода
df_income['med'] = (df_income['total_income']>=50000) & (df_income['total_income']<150000)
df_income['high'] = (df_income['total_income']>=150000)
#df_income.pivot_table(index='med', values='debt')
print(df_income.pivot_table(index='low', values='debt'),
      df_income.pivot_table(index='med', values='debt'),
     df_income.pivot_table(index='high', values='debt'))

           debt
low            
False  0.081540
True   0.062331            debt
med            
False  0.078950
True   0.083892            debt
high           
False  0.083101
True   0.079499


### Вывод

Отличается, но не существенно

- Как разные цели кредита влияют на его возврат в срок?

In [10]:
df.pivot_table(index='purpose_cat', values='debt')

,debt
purpose_cat,
автомобиль,0.093236
недвижимость,0.072590
образование,0.092971
свадьба,0.078730


### Вывод

Кредит на недвижимость и образование возрващают в срок на 2% чаще.

### 4. Итог

В данной работе проведена загрузка и изучение данных, проведена предобработка.

Исследовательский анализ вывел, что вероятность возврата в срок выше у тех, кто оформляет кредит на недвижимость, или имея низкий доход, или те, кто состоит или состоял в браке.